In [1]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1271 sha256=854421b9ec773a7f8f662b1ff631c3c13cc808f2d7c2dc27f7c1b94e39147a29
  Stored in directory: c:\users\pooja\appdata\local\pip\cache\wheels\73\2b\cb\099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [2]:
#importing libraries
import pandas as pd
import numpy as np
import sqlite3
import time
from bs4 import BeautifulSoup
import requests

ModuleNotFoundError: No module named 'pandas'

In [2]:
#reading page urls from text file
categories = pd.read_csv("Meesho_urls.txt" , sep=",")
categories.drop_duplicates(inplace = True)
all_links = categories['link']

In [3]:
len(all_links)

44

In [4]:
#creating a base csv file & adding header
import csv
try:
     with open("Meesho_product_details.csv", 'x',encoding='utf8', newline='') as csvFile:
        theWriter = csv.writer(csvFile)
        columns = ["Website", "Product_Link", "Product_Name", "Product_Brand", "Product_Category", "Size_Avail", "Price","MRP", 
                   "Gender", "Description", "Primary_Image_Links", "Secondary_Image_Links"]
        theWriter.writerow(columns)
except FileExistsError as e:
    print(e)

## Scrapping Product Details

In [5]:
start =0
end = 45
page_no = 1
#scrapping data
begin = time.time()
all_link_list = list(all_links)
all_link_list = all_link_list[start:end]
all_url_links_counts = start
print("Total Categories to scrape:-", len(all_link_list))
print()
with open("Meesho_product_details.csv", 'a', newline='',encoding='utf8') as csvFile:
    theWriter = csv.writer(csvFile)
    for cat_url in all_link_list:
        #adding page numbers to each category link
        final_url = []
        for i in range(page_no, 300):
            final_url.append(cat_url+str(i))
        count = 0
        Cat_list = categories.category[all_url_links_counts]
        Gender = categories.gender[all_url_links_counts]
        print("Scraping for catogory No.{} ==> {}".format(all_url_links_counts,Cat_list))
        print("page no....", end='')
        #scrapping for page
        for url in final_url:
            count=count+1
            try:
                page_source = requests.get(url)
                soup=BeautifulSoup(page_source.text,'html.parser')
                pager = soup.findAll('div', {'class':'sc-dkPtRN ProductList__GridCol-sc-8lnc8o-0 FjWWx jMkQHN'})
                plinks = []
                for i in pager:
                    plinks.append('https://meesho.com'+i.a['href'])
            except:
                continue
            for pro in plinks:
                try:
#                     print()
#                     print(pro)
                    containt2 = requests.get(pro, 'html.parser')
                    containt2 = BeautifulSoup(containt2.text, 'html.parser')
                    s = containt2.findAll('span',{'class':'SingleChip__StyledChip-sc-oqewox-0 gAAUDP'})
                    size = []
                    for i in s:
                        size.append(i.text)
                    size = ','.join(size)
                    if size == '':
                        size = (containt2.findAll('span', {'class':'SingleChip__StyledChip-sc-oqewox-0 fNHJlp'}))[0].text.strip()
#                     print(size)
                    pImg = containt2.findAll('div', {'class':'NewProductCarouse__ImageWrapperDesktop-sc-vkv49x-1 jWpEQd'})[0].div.img['src']
#                     print(pImg)
                    img = containt2.findAll('div',{'class':'Container-sc-wd3hnw-0 kqopSZ'})
                    ImgLinks = []
                    for i in img:
                        Img = i.div.img['src']
                        Img = Img.split('_')
                        Img[-1] = '512.jpg'
                        ImgLinks.append('_'.join(Img))
                    secImg = ImgLinks[1:]
                    secImg = ','.join(secImg)
                    if secImg == '':
                        secImg = 'Not Available'
#                     print(secImg)
                    price = containt2.find('h4', {'class':'Text__StyledText-sc-oo0kvp-0 fyTUEs'}).text.replace('₹','').strip()
#                     print(price)
                    mrp = containt2.find('p').text.replace('₹','').strip()
#                     print(mrp)
                    pName = containt2.find('span', {'class':'Text__StyledText-sc-oo0kvp-0 elstub'}).text.strip()
#                     print(pName)
                    de =containt2.findAll('p', {'class':'Text__StyledText-sc-oo0kvp-0 fCJVtz pre pre'})
                    desc = []
                    for i in de:
                        desc.append(i.text.strip().replace('\xa0:\xa0',':'))
                    desc = ' '.join(desc)
                    description = desc.replace('\xa0','')
#                     print(description)
                    pBrand = containt2.find('span', {'class':'Text__StyledText-sc-oo0kvp-0 cGaFci ShopCardstyled__ShopName-sc-du9pku-6 hHfppf ShopCardstyled__ShopName-sc-du9pku-6 hHfppf'}).text.strip()
#                     print(pBrand)
                    #writing results into csv file
                    theWriter.writerow(['https://www.meesho.com',pro,pName,pBrand,Cat_list,size,price,mrp,Gender,description,pImg,secImg])
                except Exception:
                    pass
            print(count,  end='->')
        print(end='\n\n')
        page_no = 1
        all_url_links_counts+=1
print("Completed! Total time required:",round((time.time()-begin)/60, 2),'Minutes')    

Total Categories to scrape:- 44

Scraping for catogory No.0 ==> Dresses
page no....1->2->

KeyboardInterrupt: 

# Creating Dataframe

In [6]:
#reading csv file
product_details_sel = pd.read_csv("Meesho_product_details.csv")

In [7]:
#cleaning data
product_details_sel.drop_duplicates(inplace = True)
product_details_sel.dropna(inplace = True)
product_details_sel = product_details_sel.reset_index(drop = True)
product_details_sel["Affiliate_Link"] = ''
product_details_sel.shape

(35, 13)

# DataBase

In [9]:
conn = sqlite3.connect('meeshoDb.db')
c = conn.cursor()
c.execute('CREATE TABLE if not exists product_details (Website varchar(40) NOT NULL,Product_Link TEXT PRIMARY KEY,Product_Name varchar(50) NOT NULL,Product_Brand varchar(50) NOT NULL,Product_Category varchar(50),Size_Avail varchar(20) NOT NULL,Price int NOT NULL,MRP int NOT NULL,Gender varchar(15) NOT NULL,Description TEXT NOT NULL,Primary_Image_Links TEXT NOT NULL,Secondary_Image_Links TEXT NOT NULL,Affiliate_Link TEXT NOT NULL)')
conn.commit()

In [10]:
#inserting dataframe into database
product_details_sel.to_sql('product_details', conn, if_exists='replace', index = False)

In [11]:
# Print 20 products
c.execute('''  
SELECT * FROM product_details
          ''')

for row in c.fetchmany(size=2):
    print (row)

('https://www.meesho.com', 'https://meesho.com/urbane-modern-women-dresses/p/10zs47', 'Urbane Modern Women Dresses', 'RS CREATIONSS', 'Dresses', 'XS,S,M,L,XL', 299, 399, 'Women', 'Fabric:Lycra Sleeve Length:Sleeveless Pattern:Solid Multipack:1 Sizes: S, XL, XS, L, M ', 'https://images.meesho.com/images/products/62135575/fmhli_512.jpg', 'https://images.meesho.com/images/products/62135575/izr6t_512.jpg,https://images.meesho.com/images/products/62135575/vqnv3_512.jpg,https://images.meesho.com/images/products/62135575/k8wjo_512.jpg', '')
('https://www.meesho.com', 'https://meesho.com/riyas-new-trending-westerns-for-specially-this-month/p/159eau', "RIYA'S NEW TRENDING WESTERNS FOR SPECIALLY THIS MONTH", 'POOJA SELECTION', 'Dresses', 'S,M,L,XL,XXL', 311, 411, 'Women', "Fabric:Rayon Sizes: S (Bust Size:36 in) XL (Bust Size:42 in) L (Bust Size:40 in) M (Bust Size:38 in) XXL (Bust Size:44 in)  Fabric:Rayon Sleeve Length: Short Sleeves Pattern: Dyed/ Washed Multipack: 1 Sizes: S (Length Size: 44

In [12]:
conn.close()